In [7]:


# alpha1_diag_dyad_no_trans = alpha1_diag_dyad
# alpha1_diag_dyad_no_trans[1,1] = 0
# alpha1_diag_dyad_no_trans[2,2] = 0


class NanoRod()
    fit_result_params = (
        ## eps_inf, hbar*omega_p, hbar*gamma_nr, eps_b (not used as fit param), a_x, a_yz
        15.100176  , 10.15232758/hbar,  0.10316881/hbar, 1.0, 67.24906658*nm, 19.20816015*nm
        )
    
    def __init__(self, fit_param=fit_result_params, angle=np.pi/2)
        self.fit_param = fit_param
        self.a_long = fit_param[3]
        self.a_short = fit_param[4]
        self.angle = angle
    
    def diagonal_polarizability(self):
        alpha1_diag_dyad = cp.sparse_ret_prolate_spheroid_polarizability_Drude(
            omega_drive, *fit_result_params)

    def mol_too_close(mol_x,mol_y):
        rotated_x = np.cos(self.angle)*mol_x - np.sin(self.angle)*mol_y
        rotated_y = np.sin(self.angle)*mol_x + np.cos(self.angle)*mol_y
        long_quench_radius = self.a_long + 10
        short_quench_radius = self.a_short + 10
        rotated_ellip_eq = rotated_x**2./long_quench_radius**2 + rotated_y**2./short_quench_radius**2
        if rotated_ellip_eq < 1: 
            return True
        else: 
            return False


class Molecule()
    alpha0_diag_dyad = cp.sparse_polarizability_tensor(
        mass=cp.fluorophore_mass(
            ext_coef=parameters['fluorophore']['extinction_coeff'], 
            gamma=parameters['fluorophore']['mass_gamma']/hbar
            ), 
        w_res=drive_hbar_omega/hbar, 
        w=drive_hbar_omega/hbar, 
        gamma_nr=parameters['fluorophore']['test_gamma']/hbar,
        a=0, 
        eps_inf=1, 
        ebs_b=1
        )



SyntaxError: invalid syntax (<ipython-input-7-450e8a2c248a>, line 8)

In [1]:
class TestClass:
    
    def __init__(self, test_arg):
        pass


In [5]:
test_instance = TestClass('hello')

In [6]:
test_instance

In [ ]:
class MolCoupNanoRodExp:
    ''' Collect focused+diffracted far-field information from molecules nearby a nanorod '''
    
    ## set up inverse mapping from observed -> true angle for signle molecule in the plane. 
    saved_mapping = np.loadtxt('obs_pol_vs_true_angle.txt')
    true_ord_angles, obs_ord_angles =  saved_mapping.T
    #from scipy import interpolate
    f = interpolate.interp1d(true_ord_angles,obs_ord_angles)
    f_inv = interpolate.interp1d(
        obs_ord_angles[:251],
        true_ord_angles[:251],
        bounds_error=False,
        fill_value=(0,np.pi/2)
        )

        
    def __init__(self, molecules_instance, mol_angle=0, plas_angle=np.pi/2):
        self.mol_locations = locations
        self.mol_angles = mol_angle
        self.rod_angle = plas_angle
        
        #### Filtering out molecules in region of fluorescence quenching 
        self.el_a = 19 ## ranorod degenerate radius
        self.el_c = 67
        self.quel_a = self.el_a + 10 ## define quenching region
        self.quel_c = self.el_c + 10
        self.input_x_mol = locations[:,0]
        self.input_y_mol = locations[:,1]
        self.pt_is_in_ellip = np.ones(self.input_x_mol.shape, dtype=bool) ## initialize index array
        for i in np.arange(locations[:,0].shape[0]):  
            ## currently works only if nanorod is vertically oriented
            if (self.input_x_mol[i]**2./self.quel_a**2. +  self.input_y_mol[i]**2./self.quel_c**2.) < 1:
                self.pt_is_in_ellip[i] = False
        ## select molecules outside region,
        self.mol_locations = locations[self.pt_is_in_ellip]
        ## select molecule angles if listed per molecule, 
        if type(mol_angle)==np.ndarray and mol_angle.shape[0]>1:
            self.mol_angles = mol_angle[self.pt_is_in_ellip]
        else: self.mol_angles = mol_angle
        
        (self.mol_E, 
         self.plas_E, 
         self.p0_unc_E, 
         self.p0, 
         self.p1) = dipole_fields(self.mol_locations, self.mol_angles, self.rod_angle)
        
        self.trial_images = image_from_E(self.mol_E + self.plas_E )

        self.appar_cents = calculate_apparent_centroids(self.trial_images)
        self.x_cen, self.y_cen = self.appar_cents
        
        ## Calculate polarization with beam splitter
        self.angles, self.Px_per_drive_I, self.Py_per_drive_I = powers_and_angels(
            self.mol_E + self.plas_E
            )
        self.mispol_angle= MolCoupNanoRodExp.f_inv(self.angles)

        self.default_plot_limits = [
            np.min(self.mol_locations)-((np.max(self.mol_locations)-np.min(self.mol_locations))*.1),
            np.max(self.mol_locations)+((np.max(self.mol_locations)-np.min(self.mol_locations))*.1)]
    def plot_mispol_map(self, plot_limits=None):
        if plot_limits == None: plot_limits = self.default_plot_limits
        quiv_ax, = quiver_plot(self.mol_locations[:,0], self.mol_locations[:,1], self.mispol_angle, plot_limits,
                              true_mol_angle=self.mol_angles)
        return quiv_ax
    
    def plot_mispol_map_wMisloc(self, plot_limits=None):
        if plot_limits == None: plot_limits = self.default_plot_limits
        quiv_ax = self.plot_mispol_map(plot_limits)
        scatter_centroids_wLine(self.mol_locations[:,0], self.mol_locations[:,1],
                                self.plas_E+self.mol_E, quiv_ax)
        
    def plot_mislocalization_magnitude_correlation(self):
        self.misloc_mag = calculate_mislocalization_magnitude(
            self.x_cen, self.y_cen, self.mol_locations[:,0], self.mol_locations[:,1]
            )
    
        plt.figure(dpi=300)
        plt.scatter(misloc_mag, mispol_angle, s=10, c='Black', zorder=3)
        plt.tight_layout()
        plt.xlabel('Magnitude of mislocalization [nm]')
        plt.ylabel('Apparent angle [deg]')
        plt.yticks([0,  np.pi/8,  np.pi/4, np.pi/8 *3, np.pi/2],
                   ['0','22.5','45','57.5','90'])

    